# Dimensionality Reduction - SVD

Collaborative recommendation systems employ both user-to-user and item-to-item approaches to generate personalized recommendations. These systems typically operate in two modes:
- An offline mode for computing recommendations
- An online mode for serving responses to customer queries

When analyzing the computational complexity of these algorithms:
- User-to-user comparisons: O(kn²), where n is the number of users and k represents the k-nearest neighbors
- Item-to-item comparisons: Similarly O(kn²)
- Space complexity: O(nk) for both approaches, as we need to store the top k closest matches for each user

While the offline calculation of recommendations is computationally expensive, it can be distributed across multiple machines using various parallel processing techniques. However, working with sparse rating matrices presents additional challenges in computing similarities efficiently.

To address these challenges, dimensionality reduction techniques can be employed to transform the rating matrices into a lower-dimensional space, resulting in a more compact representation. The two primary methods for this transformation are:

1. Singular Value Decomposition (SVD)
2. Principal Component Analysis (PCA)

Let's explore these concepts through a practical example.

In [15]:
import pandas as pd
import numpy as np
import scipy.stats

# Ratings of users by products, 5 users 6 products
data = {
    'movieA': [1,7,3,5,3,5,3,5,3,5,3,5],
    'movieB': [1,7,1,7,1,7,1,7,1,7,1,7],
    'movieC': [1,7,1,7,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]
}

index = [f"user-{i}" for i in range(1, 13)];
data_frame = pd.DataFrame(data, index=index).astype(float)

print("Data table prior to Pearson Correlation Coefficient calculation") 
print(data_frame)

Data table prior to Pearson Correlation Coefficient calculation
         movieA  movieB  movieC
user-1      1.0     1.0     1.0
user-2      7.0     7.0     7.0
user-3      3.0     1.0     1.0
user-4      5.0     7.0     7.0
user-5      3.0     1.0     NaN
user-6      5.0     7.0     NaN
user-7      3.0     1.0     NaN
user-8      5.0     7.0     NaN
user-9      3.0     1.0     NaN
user-10     5.0     7.0     NaN
user-11     3.0     1.0     NaN
user-12     5.0     7.0     NaN


In our dataset, we have a table showing users and their movie ratings. There's a notable gap in the data where users 5-12 haven't provided ratings for MovieC. Before we can proceed with dimensionality reduction, we need to handle these missing values.

To address these missing ratings (NaN values), we have two options:
1. Calculate the mean rating for each row (user's average rating)
2. Calculate the mean rating for each column (movie's average rating)

Given that MovieC has very few ratings (sparse column), we'll opt to use the row means - each user's average rating across other movies - to fill in the missing values.

In [17]:
data_frame = data_frame.apply(lambda row: row.fillna(row.mean()), axis=1)
print("Row adjusted mean")
print(data_frame)

Row adjusted mean
         movieA  movieB  movieC
user-1      1.0     1.0     1.0
user-2      7.0     7.0     7.0
user-3      3.0     1.0     1.0
user-4      5.0     7.0     7.0
user-5      3.0     1.0     2.0
user-6      5.0     7.0     6.0
user-7      3.0     1.0     2.0
user-8      5.0     7.0     6.0
user-9      3.0     1.0     2.0
user-10     5.0     7.0     6.0
user-11     3.0     1.0     2.0
user-12     5.0     7.0     6.0


You will need to use your imagination a bit, but imagine this list had a billion of users with hundreds of millions of movies. 

Singular Value Decomposition (SVD) is a fundamental matrix factorization technique in linear algebra and data analysis. It decomposes a matrix into three other matrices, revealing important properties of the original matrix. Some key points:

1. Decomposition: For a matrix A, SVD factorizes it into A = USV^T, where:
   - U is an orthogonal matrix of left singular vectors
   - S is a diagonal matrix of singular values
   - V^T is the transpose of an orthogonal matrix of right singular vectors
2. Dimensionality Reduction: SVD can be used to create lower-rank approximations of the original matrix by keeping only the largest singular values and their corresponding singular vectors.
3. Applications:
   - Data compression
   - Noise reduction in data
   - Recommendation systems
   - Latent semantic analysis in natural language processing
4. Properties:
   - Works on any real or complex matrix
   - Reveals the rank of the matrix
   - Useful for computing matrix inverses and pseudoinverses
5. In machine learning: SVD helps in feature extraction and dimensionality reduction, making it easier to work with high-dimensional data.
6. Computational aspects: There are efficient algorithms for computing SVD, making it practical for large datasets.

Lets compute the similarity matrix using SVD.

In [20]:
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cosine

# 1. First standardize the data (center and scale)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_frame)

# 2. Perform SVD
U, S, VT = np.linalg.svd(scaled_data, full_matrices=False)

# 3. Create the similarity matrix
# You can choose how many components to keep (k)
k = 2  # for example, keeping 2 components

# Reduce dimensions
S_reduced = np.diag(S[:k])
U_reduced = U[:, :k]
VT_reduced = VT[:k, :]

# Reconstruct the matrix with reduced dimensions
reconstructed_matrix = U_reduced @ S_reduced @ VT_reduced

# 4. Calculate similarity matrix (user-user similarities)
similarity_matrix = np.zeros((len(data_frame), len(data_frame)))
for i in range(len(data_frame)):
    for j in range(len(data_frame)):
        similarity_matrix[i][j] = 1 - cosine(reconstructed_matrix[i], reconstructed_matrix[j])

# Convert to DataFrame for better visualization
similarity_df = pd.DataFrame(
    similarity_matrix, 
    index=data_frame.index, 
    columns=data_frame.index
)

print("Similarity Matrix:")
print(similarity_df)

Similarity Matrix:
           user-1    user-2    user-3    user-4    user-5    user-6    user-7  \
user-1   1.000000 -1.000000  0.886676 -0.886676  0.903327 -0.903327  0.903327   
user-2  -1.000000  1.000000 -0.886676  0.886676 -0.903327  0.903327 -0.903327   
user-3   0.886676 -0.886676  1.000000 -1.000000  0.999302 -0.999302  0.999302   
user-4  -0.886676  0.886676 -1.000000  1.000000 -0.999302  0.999302 -0.999302   
user-5   0.903327 -0.903327  0.999302 -0.999302  1.000000 -1.000000  1.000000   
user-6  -0.903327  0.903327 -0.999302  0.999302 -1.000000  1.000000 -1.000000   
user-7   0.903327 -0.903327  0.999302 -0.999302  1.000000 -1.000000  1.000000   
user-8  -0.903327  0.903327 -0.999302  0.999302 -1.000000  1.000000 -1.000000   
user-9   0.903327 -0.903327  0.999302 -0.999302  1.000000 -1.000000  1.000000   
user-10 -0.903327  0.903327 -0.999302  0.999302 -1.000000  1.000000 -1.000000   
user-11  0.903327 -0.903327  0.999302 -0.999302  1.000000 -1.000000  1.000000   
user-12 -